In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.misc import comb
from sklearn.model_selection import train_test_split , KFold, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import pdb
from sklearn.metrics import roc_auc_score

%matplotlib inline

# Importo datos 

In [2]:
X_comp = pd.read_csv('X_competencia.csv')
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X.drop(['index'],inplace=True, axis=1)
y.drop(['index'], inplace=True,axis=1)
X.isnull().any().any()



False

In [3]:
X_desarrollo , X_holout ,y_desarrollo, y_holdout = train_test_split(X, y['output'],
                                                                    test_size=0.13,random_state=0,stratify=y['output'])
y_desarrollo_np=np.array(y_desarrollo)


### K-fold CV 

In [4]:
acc_train=[]
acc_val=[]
ROC_AUC_train=[]
ROC_AUC_val=[]


kfold = KFold(n_splits=5)
kfold.get_n_splits(X_desarrollo)
#este for itera sobre los k folds en cada loop tego un set de datos y otro de validacion
for train, test  in kfold.split(X_desarrollo):
    #print("TRAIN:", train_index,'\n', "TEST:", test_index,'\n' )
    X_train, X_val = X_desarrollo.iloc[train], X_desarrollo.iloc[test]
    y_train, y_val = y_desarrollo_np[train], y_desarrollo_np[test]
    #intancio el arbol que voy a entrenar en cada fold
    arbol = DecisionTreeClassifier(max_depth=3, criterion="gini")
    
    arbol.fit(X_train, y_train.astype(int))
    acc_train.append(arbol.score(X=X_train, y=y_train))
    acc_val.append(arbol.score(X=X_val, y=y_val))
    ROC_AUC_train.append(roc_auc_score(y_train,arbol.predict(X_train)))
    ROC_AUC_val.append(roc_auc_score(y_val,arbol.predict(X_val)))




  #pdb.set_trace()    

# Tabla de precision

In [5]:
tabla_prec = pd.DataFrame({ 'Partición' : np.arange(1,6),'Accuracy(training)' :acc_train,
                          'Accuracy(validation)' : acc_val,
                          'ROC AUC(train)' : ROC_AUC_train,
                          'ROC AUC(val)' : ROC_AUC_val})
display(tabla_prec)  

,Partición,Accuracy(training),Accuracy(validation),ROC AUC(train),ROC AUC(val)
0,1,0.833333,0.689655,0.832781,0.689011
1,2,0.824713,0.678161,0.827297,0.676471
2,3,0.816092,0.620690,0.807595,0.613468
3,4,0.841954,0.609195,0.839314,0.593137
4,5,0.807471,0.632184,0.809435,0.620915
